# Построение дашборда, используя sql-запросы

Цель: извлечение данных, используя sql-запросы.
Построение простого дашборда на основе библиотеки matplotlib

 В качестве  данных будем использовать часть базы данных из сервиса доставки продуктов в разные страны, которая хранит  детали заказа, информацию о продукте и категории товара

In [1]:
import pandas as pd                    # Загружаем необходимые библиотеки
import sqlite3
import matplotlib.pyplot as plt
import numpy as np

In [2]:
con = sqlite3.connect(r"Delivery_data.db")    # Получаем тренировчную БД
cur = con.cursor()
cur.execute('select name from sqlite_master where type = "table"').fetchall()            #  Получаем название таблиц

[('orders',), ('products',), ('categories',)]

### 1.Краткий обзор предоставленных данных

In [3]:
pd.read_sql('select * from orders limit 3',con)

,ProductID,OrderID,Quantity,UnitPrice,OrderDate,ShipCountry
0,1,10285,45,14.4,1996-08-20 00:00:00,Germany
1,1,10294,18,14.4,1996-08-30 00:00:00,USA
2,1,10317,20,14.4,1996-09-30 00:00:00,USA


In [4]:
pd.read_sql('select * from products limit 3',con)

,CategoryID,ProductID,ProductName
0,1,1,Chai
1,1,2,Chang
2,1,24,Guaraná Fantástica


In [6]:
pd.read_sql('select * from categories limit 3',con)

,CategoryID,CategoryName
0,1,Beverages
1,2,Condiments
2,3,Confections


### 2. Вычисление нужных метрик с помощью языка SQL

In [7]:
countries = pd.read_sql('''
    SELECT
        ShipCountry AS "Страна-заказчик",                                          
        ROUND(SUM(Quantity * UnitPrice),2) AS "Сумма заказов",                     
        ROUND(AVG(SUM(Quantity * UnitPrice)) OVER(),2) AS "Средняя сумма поставок" 
    FROM orders
    GROUP BY 1
    ORDER BY 2 DESC
''',con)
countries.head()

,Страна-заказчик,Сумма заказов,Средняя сумма поставок
0,USA,263566.98,64498.03
1,Germany,244640.63,64498.03
2,Austria,139496.63,64498.03
3,Brazil,114968.48,64498.03
4,France,85498.76,64498.03


1.Колонка "Страна-заказчик" выводит название страны, указанной в поле "ShipCountry".

2.Колонка "Сумма заказов" вычисляет общую сумму заказов для каждой страны-заказчика. Это делается путем умножения количества товаров на их цену и суммирования результатов. Результат округляется до двух знаков после запятой.

3.Колонка "Средняя сумма поставок" вычисляет среднюю сумму заказов для всех стран заказчиков. Это делается путем использования функции AVG, которая вычисляет среднее значение сумм заказов, рассчитанных в предыдущей колонке. Однако, чтобы вычислить это значение без деления на стран, используется OVER() без настройки.

В конце группируем по странам и сортируем по сумме заказов.

In [8]:
categories = pd.read_sql('''
    SELECT
        STRFTIME("%m",DATE(OrderDate)) AS "Месяц продажи",
        CategoryName AS "Категория продукта",
        SUM(Quantity * UnitPrice) AS "Прибыль",
        ROUND(AVG(SUM(Quantity * UnitPrice)) OVER(PARTITION BY STRFTIME("%m",DATE(OrderDate))),2) AS "Средняя прибыль за месяц"
    FROM
        orders o INNER JOIN products p
        ON o.ProductID = p.ProductID
        JOIN categories c
        ON c.CategoryID = p.CategoryID
    GROUP BY 1,2
    ORDER BY 1
''',con)
categories.head()

,Месяц продажи,Категория продукта,Прибыль,Средняя прибыль за месяц
0,01,Beverages,52782.40,20943.44
1,01,Condiments,10699.25,20943.44
2,01,Confections,21970.05,20943.44
3,01,Dairy Products,29320.00,20943.44
4,01,Grains/Cereals,17143.95,20943.44


1. `STRFTIME("%m",DATE(OrderDate)) AS "Месяц продажи"` - преобразует дату заказа в месяц и сохраняет его в столбце "Месяц продажи".

2. `SUM(Quantity * UnitPrice) AS "Прибыль"` - суммирует количество проданного товара, умноженное на цену за единицу товара, и сохраняет результат в столбце "Прибыль".

3. `ROUND(AVG(SUM(Quantity * UnitPrice)) OVER(PARTITION BY STRFTIME("%m",DATE(OrderDate))),2)"` - вычисляет среднюю прибыль за месяц с помощью функции `AVG()` вместе с `OVER()`,где составляются группы по месяцам. С помощью функции `ROUND()` округляет до 2 знаков после запятой.

4. `FROM orders o JOIN products p ON o.ProductID = p.ProductID JOIN categories c ON c.CategoryID = p.CategoryID` - выбирает таблицы "orders", "products" и "categories" и соединяет их по их соответствующим идентификаторам.

В конце группируем результаты запроса по месяцам и категориям продуктов для агрегации и сортируем по месяцам продажи.

In [10]:
products = pd.read_sql(
    '''
    SELECT
        ProductName AS "Название продукта",
        SUM(Quantity * UnitPrice) AS "Прибыль с продукта"
    FROM
        products p INNER JOIN orders o
    ON p.productID = o.productID
    GROUP BY 1
    ORDER BY 2 DESC
    LIMIT 5
    ''',con)
products

,Название продукта,Прибыль с продукта
0,Côte de Blaye,149984.2
1,Thüringer Rostbratwurst,87736.4
2,Raclette Courdavault,76296.0
3,Camembert Pierrot,50286.0
4,Tarte au sucre,49827.9


1. `ProductName AS "Название продукта"` - добавляет столбец "Название продукта", отображающий имя продукта из таблицы "products".

2. `SUM(Quantity * UnitPrice) AS "Прибыль с продукта"` - вычисляет прибыль на продукт, умножая количество проданного товара на цену за единицу товара, и сохраняет результат в столбце "Прибыль с продукта".

3. `FROM products p join orders o ON p.productID = o.productID` - выбирает таблицы "products" и "orders" и соединяет их по идентификатору продукта.

В конце ограничиваем вывод до 5 строк, чтобы увидеть самые продаваемые продукты

In [11]:
percent = pd.read_sql('''
    WITH metrics AS (
    SELECT 
        STRFTIME("%Y-%m",date(OrderDate)) AS month,
        SUM(Quantity * UnitPrice) AS Profit,
        LAG(SUM(Quantity * UnitPrice)) OVER(ORDER BY strftime("%Y-%m",date(OrderDate))) as Profit_month_ago
    FROM orders
    GROUP BY 1
    )
    SELECT
        month AS "Год,месяц",
        Profit AS "Прибыль",
        IFNULL(ROUND((Profit - Profit_month_ago) / Profit_month_ago * 100,2),0) AS  "Прирост"
    FROM metrics
    ORDER BY 1 DESC    
    ''',con)
percent.head()

,"Год,месяц",Прибыль,Прирост
0,1998-05,19898.66,-85.22
1,1998-04,134630.56,22.59
2,1998-03,109825.45,5.03
3,1998-02,104561.95,3.68
4,1998-01,100854.72,30.17


1. `WITH metrics AS (...)` - создает временную таблицу "metrics", которая используется в последующем запросе. Эта таблица содержит месяц, прибыль и прирост.

2. `STRFTIME("%Y-%m",date(OrderDate))` - извлекает из даты год и месяц
3. `LAG(SUM(Quantity * UnitPrice)) OVER(ORDER BY strftime("%Y-%m",date(OrderDate)))` - с помощью оконной функции `LAG()` выводит сумму прибыли за предыдущий месяц

3. `IFNULL(ROUND((Profit - Profit_month_ago) / Profit_month_ago * 100,2),0)` - выбирает столбцы из временной таблицы "metrics" и вычисляет прирост прибыли в процентах по отношению к прошедшему месяцу. `IFNULL()` используется для того, чтобы обработать ситуации, когда прибыль в предыдущем месяце равна нулю.

### 4. Составление дашборда, используя библиотеку matplotlib

In [12]:
fig,ax = plt.subplots(nrows=2,ncols=2,figsize=(16,10),dpi=1000)

ax[0][0].barh(countries['Страна-заказчик'],countries['Сумма заказов'],color=np.random.rand(20,3))
ax[0][0].set_xticks(np.arange(0,300_000,30_000))
ax[0][0].axvline(x=countries["Средняя сумма поставок"].unique(), color='black', label='Средняя сумма поставок',linestyle='--')
ax[0][0].legend()
ax[0][0].set_title('Суммарная выручка по странам',fontsize=14)
ax[0][0].set_facecolor('mintcream')
ax[0][0].grid(axis='x',color='black',alpha=0.2)
ax[0][0].text(x=66000,y='Poland',s='64498',fontsize=10)
# for i in list(zip(countries['Страна-заказчик'],countries['Сумма заказов'])):
#     ax[0][0].text(x=i[1]+2000,y=i[0],s=str(i[1]),fontsize=5)


ax[0][1].pie(
    x=list(products['Прибыль с продукта']),
    labels=list(products['Название продукта']),
    explode=[0.01 for i in range(5)],
    autopct= lambda x: '{:.0f}'.format(x*np.array(products['Прибыль с продукта']).sum() / 100)
#     autopct='%1.1f%%'
)
ax[0][1].set_title('Топ-5 продаваемых продуктов',fontsize=14)
ax[0][1].set_facecolor('mintcream')


x = np.arange(1,13)
lab_cat = categories['Категория продукта'].unique()

# for i,j in enumerate(categories['Категория продукта'].unique()):
#     y = np.array(categories[categories["Категория продукта"] == j].Прибыль.astype('int'))
#     print(f'y{i+1} = {y}')

y1 = [52782, 38959, 39756, 31847, 20251, 3602, 11781, 11190, 11330, 17836, 23746, 23443]  # Это выручки по каждой категории
y2 = [10699, 13345, 12924, 16640, 6802, 1886, 8976, 6835, 5612, 11357, 10773, 7839]       # за каждый месяц
y3 = [21970, 17439, 27627, 20898, 10456, 2494, 13229, 13250, 13643, 11848, 8498, 15742]
y4 = [29320, 16893, 24835, 43099, 12127, 9507, 20955, 12293, 16344, 21862, 21335, 22755]
y5 = [17143, 9171, 6860, 12344, 6972, 6882, 6053, 5799, 6574, 5893, 8865, 8163]
y6 = [13378, 32182, 7475, 29459, 7664, 5006, 7624, 8750, 16945, 18617, 6955, 24129]
y7 = [4534, 3901, 16844, 21210, 4814, 6231, 5813, 5267, 4052, 10189, 5681, 16728]
y8 = [17718, 13875, 13481, 14830, 7633, 3477, 11222, 13202, 12866, 13927, 9760, 9626]

labels = [str(i) for i in x]
width = 0.1

ax[1][0].bar(x-width*3.5, y1, width,label=lab_cat[0])
ax[1][0].bar(x-width*2.5, y2, width,label=lab_cat[1])
ax[1][0].bar(x-width*1.5, y3, width,label=lab_cat[2])
ax[1][0].bar(x-width*0.5, y4, width,label=lab_cat[3])
ax[1][0].bar(x+width*0.5, y5, width,label=lab_cat[4])
ax[1][0].bar(x+width*1.5, y6, width,label=lab_cat[5])
ax[1][0].bar(x+width*2.5, y7, width,label=lab_cat[6])
ax[1][0].bar(x+width*3.5, y8, width,label=lab_cat[7])

ax[1][0].plot(
    np.arange(1,13),
    [20943, 18221, 18725, 23791,  9590,  4886, 10707,  9573, 10921,13941, 11952, 16053], # Это средняя выручка за 1 месяц
    color='black',
    linewidth=2,
    label='Средняя прибыль',
    linestyle='--'
)

ax[1][0].set_xticks(x, labels)
# ax[1][0].set_xlabel("Месяц")
# ax[1][0].set_ylabel("Выручка")
ax[1][0].set_title("Динамика продаж категорий продуктов по месяцам",fontsize=14)
ax[1][0].legend(loc=1,prop={'size':9},ncol=2,facecolor='whitesmoke',fontsize=10,edgecolor='mistyrose')
ax[1][0].set_facecolor('mintcream')
ax[1][0].grid(axis='y',color='black',alpha=0.2)


ax[1][1].barh(percent['Год,месяц'],percent['Прирост'],color=np.random.rand(23,3))
ax[1][1].set_xticks(np.arange(-100,100,10),rotation=45)
ax[1][1].set_title("Прирост выручки по месяцам",fontsize=16)
ax[1][1].set_facecolor('mintcream')
ax[1][1].grid(axis='y',color='black',alpha=0.1)
for i in list(zip(percent['Год,месяц'],percent['Прирост'])):
    if i[1] > 0:
        ax[1][1].text(x=i[1]+2,y=i[0],s=str(i[1]),fontsize=8)
    elif i[1] < 0:
        ax[1][1].text(x=i[1]-13,y=i[0],s=str(i[1]),fontsize=8)
        
        
        
fig.set_facecolor('floralwhite')
fig.suptitle("Дашборд: Основная информация по данным",fontsize=18,)
fig.subplots_adjust(wspace=0.2,hspace=0.2)
plt.show()

fig.savefig('Dashboard')

Первый график показывает суммарную выручку по странам, используя столбчатую диаграмму,а линия показывает среднюю сумму поставок. 

Второй график - круговая диаграмма топ-5 продуктов по выручке с отображением прибыли каждого продукта.

Третий график - график с несколькими столбцами, который показывает выручку по категориям продуктов за каждый месяц. Также есть линейная диаграмма для отображения средней выручки за месяц. 

Четвертый график показывает прирост выручки в процентах по месяцам в виде горизонтальной столбчатой диаграммы. 